# csv処理

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.image as mpimg
import optuna
from itertools import cycle
import seaborn as sns
import statsmodels.api as sm 
from scipy.interpolate import interp1d
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
from sklearn.metrics import mean_squared_error
import time

pd.set_option('display.max_columns', 50)
plt.style.use('bmh')
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [5]:
%%time
oj = os.path.join
INPUT_DIR = '../input/m5-forecasting-accuracy/'
train = pd.read_csv(oj(INPUT_DIR, 'trainset.csv'))
price = pd.read_csv(oj(INPUT_DIR, 'sell_prices.csv'))
calendar = pd.read_csv(oj(INPUT_DIR, 'calendar.csv'))

sample_submit = pd.read_csv(oj(INPUT_DIR, 'submit.csv'))

CPU times: user 8.58 s, sys: 1.89 s, total: 10.5 s
Wall time: 12.6 s


In [15]:
OUTPUT_DIR = '../output/'
df = pd.read_csv(oj(OUTPUT_DIR, 'submit1_edited.csv'))
print(df.describe())

             d_1928        d_1929        d_1930        d_1931        d_1932  \
count  10164.000000  10164.000000  10164.000000  10164.000000  10164.000000   
mean       1.120917      0.970091      0.936934      0.944116      1.053621   
std        2.858058      2.341900      2.262916      2.293016      2.639846   
min       -5.000000     -7.000000     -6.000000     -5.000000     -7.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        1.000000      1.000000      1.000000      1.000000      1.000000   
max       85.000000     67.000000     67.000000     69.000000     86.000000   

             d_1933        d_1934        d_1935        d_1936        d_1937  \
count  10164.000000  10164.000000  10164.000000  10164.000000  10164.000000   
mean       1.249902      1.321921      1.078217      0.981700      0.957104   
std        3.189006      3.217809      2.565923    

In [16]:
result_column_names = df.columns[df.columns.get_loc('d_1928'):].to_list()
print(result_column_names)
df[result_column_names] = df[result_column_names].map(lambda x: max(0, x))

print(df.head())
print(df.describe())

['d_1928', 'd_1929', 'd_1930', 'd_1931', 'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
                                id          item_id      dept_id     cat_id  \
0  HOUSEHOLD_1_462_CA_4_evaluation  HOUSEHOLD_1_462  HOUSEHOLD_1  HOUSEHOLD   
1  HOUSEHOLD_1_463_CA_4_evaluation  HOUSEHOLD_1_463  HOUSEHOLD_1  HOUSEHOLD   
2  HOUSEHOLD_1_464_CA_4_evaluation  HOUSEHOLD_1_464  HOUSEHOLD_1  HOUSEHOLD   
3  HOUSEHOLD_1_465_CA_4_evaluation  HOUSEHOLD_1_465  HOUSEHOLD_1  HOUSEHOLD   
4  HOUSEHOLD_1_466_CA_4_evaluation  HOUSEHOLD_1_466  HOUSEHOLD_1  HOUSEHOLD   

  store_id state_id  d_1928  d_1929  d_1930  d_1931  d_1932  d_1933  d_1934  \
0     CA_4       CA       0       1       0       0       0       1       1   
1     CA_4       CA       0       0       0       0       0       0       0   
2     CA_4       CA       0       0       0       0       0       0       0   
3     CA_4       CA       3       2       2       2       2       3 